# <b><span style='color:#F1A424'>|</span> AES-2: <span style='color:#F1A424'>Multi-class Classification </span><span style='color:#ABABAB'> [Inference]</span></b>

***

- [Train code](https://www.kaggle.com/code/alejopaullier/aes-2-multi-class-classification-train) 👈🏼

<div class="alert alert-block alert-warning">  
<b>Note:</b> Work in progress! ⚠️ 
</div>

### <b><span style='color:#F1A424'>Table of Contents</span></b> <a class='anchor' id='top'></a>
<div style=" background-color:#3b3745; padding: 13px 13px; border-radius: 8px; color: white">
<li> <a href="#introduction">Introduction</a></li>
<li> <a href="#install_libraries">Install libraries</a></li>
<li><a href="#import_libraries">Import Libraries</a></li>
<li><a href="#configuration">Configuration</a></li>
<li><a href="#utils">Utils</a></li>
<li><a href="#load_data">Load Data</a></li>
<li><a href="#tokenizer">Tokenizer</a></li>
<li><a href="#dataset">Dataset</a></li>
<li><a href="#model">Model</a></li>
<li><a href="#inference_function">Inference Function</a></li>
<li><a href="#inference">Inference</a></li>
<li><a href="#submission">Save Submission</a></li>
</div>


# <b><span style='color:#F1A424'>|</span> Introduction</b><a class='anchor' id='introduction'></a> [↑](#top)

***

### <b><span style='color:#F1A424'>Useful References</span></b>

- [Deberta v3 Hugging Face](https://huggingface.co/microsoft/deberta-v3-base)
- [Deberta paper](https://arxiv.org/abs/2006.03654)

# <b><span style='color:#F1A424'>|</span> Import Libraries</b><a class='anchor' id='import_libraries'></a> [↑](#top)

***

Import all the required libraries for this notebook.

In [ ]:
import ast
import copy
import gc
import itertools
import joblib
import json
import math
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import os
import pandas as pd
import pickle
import random
import re
import scipy as sp
import string
import sys
import time
import torch
import torch.nn as nn
import warnings


from sklearn.metrics import cohen_kappa_score
from torch.nn import Parameter
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm

# ======= OPTIONS =========
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Current device is: {device}")
warnings.filterwarnings("ignore")
!mkdir output

### <b><span style='color:#F1A424'>Tokenizers and transformers</span></b>

In [ ]:
import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")

# <b><span style='color:#F1A424'>|</span> Configuration</b><a class='anchor' id='configuration'></a> [↑](#top)

***

Central repository for this notebook's hyperparameters.

In [ ]:
class config:
    APEX = True # Automatic Precision Enabled
    BATCH_SIZE_TEST = 16
    DEBUG = False
    GRADIENT_CHECKPOINTING = True
    MAX_LEN = 512
    MODEL = "microsoft/deberta-v3-base"
    NUM_CLASSES = 6
    NUM_WORKERS = 0 #multiprocessing.cpu_count()
    PRINT_FREQ = 20
    SEED = 20


class paths:
    BEST_MODEL_PATH = "/kaggle/input/aes2-trained-models"
    OUTPUT_DIR = "/content/output"
    SUBMISSION_CSV = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv"
    TEST_CSV = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv"
    TRAIN_CSV = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv"

# <b><span style='color:#F1A424'>|</span> Utils</b><a class='anchor' id='utils'></a> [↑](#top)

***

Utility functions used throughout the notebook.

In [ ]:
def get_score(y_true, y_pred):
    score = cohen_kappa_score(y_true, y_pred, weights='quadratic')
    return score


def seed_everything(seed=20):
    """Seed everything to ensure reproducibility"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


def sep():
    print("-"*100)

seed_everything(seed=config.SEED)

# <b><span style='color:#F1A424'>|</span> Load Data</b><a class='anchor' id='load_data'></a> [↑](#top)

***

Load data.

In [ ]:
test_df = pd.read_csv(paths.TEST_CSV, sep=',')
print(f"Test dataframe has shape: {test_df.shape}"), sep()
display(test_df.head())

# <b><span style='color:#F1A424'>|</span> Tokenizer</b><a class='anchor' id='tokenizer'></a> [↑](#top)

***

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(paths.BEST_MODEL_PATH + "/tokenizer")
vocabulary = tokenizer.get_vocab()
total_tokens = len(vocabulary)
print("Total number of tokens in the tokenizer:", total_tokens)
print(tokenizer)

# <b><span style='color:#F1A424'>|</span> Dataset</b><a class='anchor' id='dataset'></a> [↑](#top)

***

    
We need to get the `max_len` from our `tokenizer`. We create a `tqdm` iterator and for each text we extract the tokenized length. Then we get the maximum value and we add 3 for the special tokens `CLS`, `SEP`, `SEP`.

- [Hugging Face Padding and Truncation](https://huggingface.co/docs/transformers/pad_truncation): check truncation to `max_length` or `True` (batch max length).

In [ ]:
lengths = []
tqdm_loader = tqdm(test_df['full_text'].fillna("").values, total=len(test_df))
for text in tqdm_loader:
    length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
    lengths.append(length)

# config.MAX_LEN = max(lengths) + 3 # cls & sep & sep
_ = plt.hist(lengths, bins=25)

In [ ]:
def prepare_input(cfg, text, tokenizer):
    """
    This function tokenizes the input text with the configured padding and truncation. Then,
    returns the input dictionary, which contains the following keys: "input_ids",
    "token_type_ids" and "attention_mask". Each value is a torch.tensor.
    :param cfg: configuration class with a TOKENIZER attribute.
    :param text: a numpy array where each value is a text as string.
    :return inputs: python dictionary where values are torch tensors.
    """
    inputs = tokenizer.encode_plus(
        text,
        return_tensors=None,
        add_special_tokens=True,
        max_length=cfg.MAX_LEN,
        padding='max_length', # TODO: check padding to max sequence in batch
        truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long) # TODO: check dtypes
    return inputs


def collate(inputs):
    """
    It truncates the inputs to the maximum sequence length in the batch.
    """
    mask_len = int(inputs["attention_mask"].sum(axis=1).max()) # Get batch's max sequence length
    for k, v in inputs.items():
        inputs[k] = inputs[k][:,:mask_len]
    return inputs


class CustomDataset(Dataset):
    def __init__(self, cfg, df, tokenizer):
        self.cfg = cfg
        self.tokenizer = tokenizer
        self.texts = df['full_text'].values
        self.essay_ids = df['essay_id'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        output = {}
        output["inputs"] = prepare_input(self.cfg, self.texts[item], self.tokenizer)
        output["essay_ids"] = self.essay_ids[item]
        return output

# <b><span style='color:#F1A424'>|</span> Model</b><a class='anchor' id='model'></a> [↑](#top)

***

In [ ]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings


class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        # Load config by inferencing it from the model name.
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.MODEL, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
        # Load config from a file.
        else:
            self.config = torch.load(config_path)

        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.MODEL, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)

        if self.cfg.GRADIENT_CHECKPOINTING:
            self.model.gradient_checkpointing_enable()

        # Add MeanPooling and Linear head at the end to transform the Model into a RegressionModel
        self.pool = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, config.NUM_CLASSES)

    def _init_weights(self, module):
        """
        This method initializes weights for different types of layers. The type of layers
        supported are nn.Linear, nn.Embedding and nn.LayerNorm.
        """
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def feature(self, inputs):
        """
        This method makes a forward pass through the model, get the last hidden state (embedding)
        and pass it through the MeanPooling layer.
        """
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        feature = self.pool(last_hidden_states, inputs['attention_mask'])
        return feature

    def forward(self, inputs):
        """
        This method makes a forward pass through the model, the MeanPooling layer and finally
        then through the Linear layer to get a regression value.
        """
        feature = self.feature(inputs)
        output = self.fc(feature)
        return output

# <b><span style='color:#F1A424'>|</span> Inference Function</b><a class='anchor' id='inference_function'></a> [↑](#top)

***

In [ ]:
def inference_fn(config, test_df, tokenizer, device):
    # ======== DATASETS ==========
    test_dataset = CustomDataset(config, test_df, tokenizer)

    # ======== DATALOADERS ==========
    test_loader = DataLoader(
        test_dataset,
        batch_size=config.BATCH_SIZE_TEST,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        drop_last=False
    )
    
    # ======== MODEL ==========
    softmax = nn.Softmax(dim=1)
    model = CustomModel(config, config_path=paths.BEST_MODEL_PATH + "/config.pth", pretrained=False)
    state = torch.load(paths.BEST_MODEL_PATH + "/best_model.pth")
    model.load_state_dict(state["model"])
    model.to(device)
    model.eval() # set model in evaluation mode
    prediction_dict = {}
    preds = []
    idx = []
    with tqdm(test_loader, unit="test_batch", desc='Inference') as tqdm_test_loader:
        for step, batch in enumerate(tqdm_test_loader):
            inputs = batch.pop("inputs")
            ids = batch.pop("essay_ids")
            inputs = collate(inputs) # collate inputs
            for k, v in inputs.items():
                inputs[k] = v.to(device) # send inputs to device
            with torch.no_grad():
                y_preds = model(inputs) # forward propagation pass
                _, y_preds = torch.max(softmax(torch.tensor(y_preds)), dim=1)
            preds.append(y_preds.to('cpu').numpy()) # save predictions
            idx.append(ids)

    prediction_dict["predictions"] = np.concatenate(preds)
    prediction_dict["essay_ids"] = np.concatenate(idx)
    return prediction_dict

# <b><span style='color:#F1A424'>|</span> Inference</b><a class='anchor' id='inference'></a> [↑](#top)

***

In [ ]:
predictions = inference_fn(config, test_df, tokenizer, device)

# <b><span style='color:#F1A424'>|</span> Save Submission</b><a class='anchor' id='submission'></a> [↑](#top)

***

In [ ]:
submission = pd.DataFrame()
submission["essay_id"] = predictions["essay_ids"]
submission["score"] = predictions["predictions"]
submission["score"] = submission["score"] + 1 
print(f"Submission shape: {submission.shape}")
submission.to_csv("submission.csv", index=False)
submission